# <h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

# 1. Dataframe Creating

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [10]:
#Import libraries that we'll need

import pandas as pd # library for data analsysis
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
import numpy as np
import json # library to handle JSON files
#!conda install -c conda-forge lxml --yes
#!conda install -c conda-forge requests beautifulsoup4 --yes

#!pip install geocoder
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import requests



print('Libraries imported.')

Libraries imported.


[]

In [36]:
import bs4 as bs
import urllib.request
#List where html table rows will be append
Trt_table=[]

#Get html page containing the table to extract
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'lxml')
#Detecting all tables presnt in the page
table = soup.findAll("table", {"class":"wikitable"})[0]

#Finding table entire rows
table_rows = table.find_all('tr')

#Table row reading and appending in the list difined above
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    Trt_table.append(row)

#Convert list to dataframe
df= pd.DataFrame(Trt_table)

#Dataframe columns defining
df.columns =['PostalCode','Borough','Neighborhood']

#Dataframe wrangling
df['PostalCode']= df['PostalCode'].str[:-1]
df['Borough']= df['Borough'].str[:-1]
df['Neighborhood']= df['Neighborhood'].str[:-1]
df.replace('Not assigned', np.nan, inplace = True)
df=df.drop([0], axis=0)
df.dropna(subset=['Borough'], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)
print('Dataframe shape: ',df.shape)
df

Dataframe shape:  (103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# 2. Dataframe geographical coordinates adding

## Lets download the dataset

In [24]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

--2021-02-26 10:50:07--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 169.63.96.176, 169.63.96.194
Connecting to cocl.us (cocl.us)|169.63.96.176|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-02-26 10:50:08--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-02-26 10:50:08--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1

In [35]:
#Load data from csv file
lat_long = pd.read_csv('Geospatial_Coordinates.csv')

#Rename postal code column for merging with df 
lat_long.rename(columns={'Postal Code':'PostalCode'}, inplace=True)



## Merge borough data frame with their geographical coordinates using PostalCode as common column

In [34]:
Toronto_df= pd.merge(df,lat_long,on='PostalCode')
Toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
